In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mlxtend.classifier import EnsembleVoteClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
# Read the dataset
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')

def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

def get_vector(df, train_mode):
    if train_mode:
        df['tokens'] = df['facts'].apply(tokenize)
    else:
        df['tokens'] = df['facts'].apply(tokenize)
    
    tokenized_text = df['tokens'].apply(lambda x: ' '.join(x))
    X_party1 = df['first_party']
    X_party2 = df['second_party']

    X = np.concatenate([X_party1.values.reshape(-1, 1), X_party2.values.reshape(-1, 1), tokenized_text.values.reshape(-1, 1)], axis=1)
    return X

X_train = get_vector(train, True)
y_train = train["first_party_winner"]
X_test = get_vector(test, False)

In [42]:
# Tokenize text using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_text = vectorizer.fit_transform(X_train[:, 2])
X_test_text = vectorizer.transform(X_test[:, 2])

ct = ColumnTransformer([('encoder', OneHotEncoder(), [0, 1])], remainder='passthrough')
X_train_categorical = ct.fit_transform(X_train[:, :2])
X_test_categorical = ct.fit_transform(X_test[:, :2])


In [56]:
# Combine the encoded categorical features with the text features
X_train_encoded = np.concatenate([X_train_categorical.toarray(), X_train_text.toarray()], axis=1)
X_test_encoded = np.concatenate([X_test_categorical.toarray(), X_test_text.toarray()], axis=1)

# Split the data into train and validation sets
X_train_encoded, X_val_encoded, y_train1, y_val = train_test_split(X_train_encoded, y_train, test_size=0.2, random_state=42)


In [20]:
# Define the base models
base_models = [
    ('rf', RandomForestClassifier()),
    ('lr', LogisticRegression()),
    ('lgbm', LGBMClassifier()),
    ('xgb', XGBClassifier())
]

# Initialize the EnsembleVoteClassifier
ensemble = EnsembleVoteClassifier(clfs=[model for _, model in base_models], voting='soft')

# Train the base models
for name, model in base_models:
    model.fit(X_train_encoded, y_train1)
    y_pred_val = model.predict(X_val_encoded)
    accuracy = accuracy_score(y_val, y_pred_val)
    print(f"{name} Validation Accuracy: {accuracy}")

rf Validation Accuracy: 0.655241935483871
lr Validation Accuracy: 0.6532258064516129
lgbm Validation Accuracy: 0.594758064516129
xgb Validation Accuracy: 0.6209677419354839


In [21]:
# Train the ensemble model
ensemble.fit(X_train_encoded, y_train1)
y_pred_val_ensemble = ensemble.predict(X_val_encoded)
accuracy_ensemble = accuracy_score(y_val, y_pred_val_ensemble)
print(f"Ensemble Validation Accuracy: {accuracy_ensemble}")

Ensemble Validation Accuracy: 0.6391129032258065


In [58]:
# Predict on the test set using the ensemble model
y_pred_test = ensemble.predict(X_test_encoded)

# Prepare submission DataFrame
submission = pd.DataFrame({'ID': test['ID'], 'first_party_winner': y_pred_test})

# Save the submission file
submission.to_csv('ensemble_nltk_submission.csv', index=False)
print("Submission file saved.")


ValueError: X has 19571 features, but RandomForestClassifier is expecting 21537 features as input.

In [26]:
X_test_encoded.shape

(1240, 11160)

In [28]:
X_train_encoded.shape

(1982, 21537)

In [29]:
X_val_encoded.shape

(496, 21537)

In [31]:
X_train.shape

(2478, 3)

In [33]:
X_test.shape

(1240, 3)